In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# 1. Create new Model OrderForm
# 2. Insert new OrderForm record
# 3. Read file from hardcoded location
# 4. Use Openpyxl to extract order details
# 5. Use Openpyxl to extract product count for order
# 6. Create new Order model
# 7. Any exceptions because of failed validation, create new record in OrderFormReaderException


In [2]:
# 0. Setup
import os
import json
from pathlib import Path
import sys
from openpyxl import load_workbook
from django.conf import settings

filename = 'sample-order-v070420.xlsx'
django_project_relative_path = os.path.join('order',filename)
xlsx_file = os.path.join(settings.PROJECT_DIR,django_project_relative_path)
xlsx_file

'/home/jon/code/playground/farmbox/api/order/sample-order-v070420.xlsx'

In [3]:
# 1. Create the OrderForm instance
from django.utils import timezone
from order.models import OrderForm, FulfillmentEvent

f_event = FulfillmentEvent.objects.last()

frm = OrderForm(filename=filename, fulfillment_event=f_event, created_at=timezone.now())
frm.save()


In [4]:
# Read XLSX file
workbook = load_workbook(filename=xlsx_file, read_only=True)
worksheet = workbook.active
excel_data = list(worksheet.rows)
# print(excel_data)

In [5]:
# 4. Crudely capture the order details
order_details_column = 'C'
order_detail_cells ={
    'C2' : 'customer_name',
    'C3': 'customer_address',
    'C4': 'customer_postcode',
    'C5': 'customer_email',
    'C6': 'fulfillment_method',
    'C7': 'collection_method',
    'C8': 'fulfillment_event__target_date'
}
captured_order_details = {}
for row in excel_data:
    for cell in row:
        try:
            captured_order_details[order_detail_cells.get(cell.coordinate,'')] = cell.internal_value
#             print(order_detail_cells.get(cell.coordinate,''))
        except Exception:
            pass

captured_order_details

{'': None,
 'customer_name': 'Nigel Samplestock',
 'customer_address': '107, Fairfield Drive',
 'customer_postcode': 'rh4 1jj',
 'customer_email': 'Dev-farmbox99@howapped.com',
 'fulfillment_method': '0789 449 542',
 'collection_method': 'Collection',
 'fulfillment_event__target_date': 'Denbies'}

In [26]:
# 5. Capture the products and desired count

# captured_product_counts = {
#     'Village Greens Veg Bag : Medium' : 1,
#     'Butternut Squash' : 1
# }
from product.models import Product
captured_product_counts = {}
product_name_column = 'A'
product_count_column = 'C'

# get list of product names
products = list(Product.objects.values_list('name',flat=True))
products_rows = []
# print(products)



# first pass to capture all cell.row where there are valid products
for row in excel_data:
    for cell in row:
        if cell.value == None:
            break
        if cell.column_letter == product_name_column and cell.internal_value in products:
            products_rows.append(row)
#             continue 2
                
                
for row in products_rows:
    for cell in row:
        if cell.column_letter == product_name_column:
            key = cell.internal_value
        if cell.column_letter == product_count_column:
            value = cell.internal_value
        captured_product_counts[key] = value
# print(captured_product_counts)

# loop over rows to capture where count is not zero

print(captured_product_counts)


{'Village Greens Veg Bag : Large': 1, 'Village Greens Veg Bag : Medium': None, 'Village Greens Veg Bag : Small': None, 'Fine green beans': None, 'Broccoli': None, 'Butternut Squash': 1, 'Cabbage Hispi': None, 'Cabbage Savoy': None, 'Cabbage Red': None, 'Carrots : loose washed': None, 'Cauliflower': None}


In [ ]:
# Create order model with m2m products

# If there's an exception, create instance `OrderFormReaderException'